In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import PercentFormatter
from matplotlib.colors import Normalize

import re

import folium
from folium import plugins

In [2]:
# 폰트 설정
import matplotlib as mpl
import matplotlib.font_manager as fm

font_path = r"C:\Windows\Fonts\malgun.ttf"  
font_prop = fm.FontProperties(fname=font_path)
mpl.rcParams['font.family'] = font_prop.get_name()
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
# data load
df = pd.read_csv(r'C:\\Users\\user\\Desktop\\연구\\5. 국방부 용역과제\\250730_v2.csv', encoding = 'cp949')
df_geo = pd.read_csv(r'..\\datasets\\df_geo_new_v2.csv') # 위경도 좌표

print(df.head())
print(df_geo.head())

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   
3   4        서울특별시 강남구 학동로 432       경기도 과천시 중앙로 294      경기도 과천시 중앙로 294   
4   5         대구광역시 남구 명덕로 236      대구광역시 수성구 무열로 56     대구광역시 수성구 무열로 56   

    중간집결 기한    최종도착 기한 이동방법  
0  0+1일 14시   0+1일 14시   개별  
1  0+28일 9시  0+28일 14시   집단  
2   0+3일 8시   0+3일 14시   집단  
3  0+1일 14시   0+1일 14시   개별  
4  0+3일 14시   0+3일 14시   개별  
   num  start_lat   start_lng    mid_lat     mid_lng    end_lat     end_lng
0    1  37.626949  126.827581  37.713020  126.902443  37.713020  126.902443
1    2  35.881731  128.567540  35.839566  129.185196  37.713651  126.889915
2    3  37.540484  126.867756  37.506451  127.012692  37.503393  129.130509
3    4  37.517981  127.048764  37.441493  126.996973 

In [4]:
# Merge
df_merge = pd.merge(df, df_geo, left_on = '순번', right_on = 'num', how = 'left').drop(columns = ['num'])

print(df_merge.head(3))
print(df_merge.tail(3))

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   

    중간집결 기한    최종도착 기한 이동방법  start_lat   start_lng    mid_lat     mid_lng  \
0  0+1일 14시   0+1일 14시   개별  37.626949  126.827581  37.713020  126.902443   
1  0+28일 9시  0+28일 14시   집단  35.881731  128.567540  35.839566  129.185196   
2   0+3일 8시   0+3일 14시   집단  37.540484  126.867756  37.506451  127.012692   

     end_lat     end_lng  
0  37.713020  126.902443  
1  37.713651  126.889915  
2  37.503393  129.130509  
            순번                 출발지 주소             중간집결지 주소  \
880064  880065  강원특별자치도 원주시 단관공원길 111  강원특별자치도 원주시 상지대길 83   
880065  880066  강원특별자치도 원주시 단관공원길 111  강원특별자치도 원주시 상지대길 83   
880066  880067  강원특별자치도 원주시 단관공원길 111  강원특별자치도 원주시 상지대길 83   

                       최종

In [5]:
df_merge = df_merge.dropna().reset_index(drop = True)

In [6]:
# statistics
print(df_merge.shape)
print(df_merge['이동방법'].value_counts())

# 위경도 NA 확인
num_na = df_merge.isna().any(axis = 1).sum()
mask = df_merge.notna().all(axis = 1)
print('num_na:', num_na)

df_merge = df_merge[mask].reset_index(drop = True)
print(df_merge)
print(df_merge['이동방법'].value_counts())
# 변환안된 것: 5,067행
# 사용가능 총: 810,708행

# # 저장
# df_merge.to_csv('bottle.csv', index = False, encoding = 'cp949')

(874983, 13)
이동방법
개별    485554
집단    389429
Name: count, dtype: int64
num_na: 0
            순번                   출발지 주소              중간집결지 주소  \
0            1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142   
1            2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516   
2            3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26   
3            4        서울특별시 강남구 학동로 432       경기도 과천시 중앙로 294   
4            5         대구광역시 남구 명덕로 236      대구광역시 수성구 무열로 56   
...        ...                      ...                   ...   
874978  880063    강원특별자치도 원주시 단관공원길 111   강원특별자치도 원주시 상지대길 83   
874979  880064    강원특별자치도 원주시 단관공원길 111   강원특별자치도 원주시 상지대길 83   
874980  880065    강원특별자치도 원주시 단관공원길 111   강원특별자치도 원주시 상지대길 83   
874981  880066    강원특별자치도 원주시 단관공원길 111   강원특별자치도 원주시 상지대길 83   
874982  880067    강원특별자치도 원주시 단관공원길 111   강원특별자치도 원주시 상지대길 83   

                       최종 도착지 주소   중간집결 기한    최종도착 기한 이동방법  start_lat  \
0            경기도 고양시 덕양구 혜음로 142  0+1일 14시   0+1일 14시   개별  37.626

In [7]:
# 시간 변환
# 중간시간 변환
df_merge_mid = df_merge['중간집결 기한'].str.split('일', expand = True)
df_merge_mid.columns = ['Mday_mid', 'Mhour_mid']
df_merge_mid['Mday_mid'] = df_merge_mid['Mday_mid'].str.replace('0+', '').astype(int)
df_merge_mid['Mhour_mid'] = df_merge_mid['Mhour_mid'].str.replace('시', '').astype(int)
df_merge_mid['mid_td'] = (pd.to_timedelta(df_merge_mid['Mday_mid'], unit = 'D') + pd.to_timedelta(df_merge_mid['Mhour_mid'], unit = 'h'))

# 최종시간 변환
df_merge_end = df_merge['최종도착 기한'].str.split('일', expand = True)
df_merge_end.columns = ['Mday_end', 'Mhour_end']
df_merge_end['Mday_end'] = df_merge_end['Mday_end'].str.replace('0+', '').astype(int)
df_merge_end['Mhour_end'] = df_merge_end['Mhour_end'].str.replace('시', '').astype(int)
df_merge_end['end_td'] = (pd.to_timedelta(df_merge_end['Mday_end'], unit = 'D') + pd.to_timedelta(df_merge_end['Mhour_end'], unit = 'h'))

df_merge = pd.concat([df_merge, df_merge_mid, df_merge_end], axis = 1).drop(columns = ['중간집결 기한', '최종도착 기한', 'Mday_mid', 'Mhour_mid', 'Mday_end', 'Mhour_end']).reset_index(drop = True)
print(df_merge.head(3))

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   

  이동방법  start_lat   start_lng    mid_lat     mid_lng    end_lat     end_lng  \
0   개별  37.626949  126.827581  37.713020  126.902443  37.713020  126.902443   
1   집단  35.881731  128.567540  35.839566  129.185196  37.713651  126.889915   
2   집단  37.540484  126.867756  37.506451  127.012692  37.503393  129.130509   

            mid_td           end_td  
0  1 days 14:00:00  1 days 14:00:00  
1 28 days 09:00:00 28 days 14:00:00  
2  3 days 08:00:00  3 days 14:00:00  


In [8]:
# M+7일 이내 움직이는 사람만
df_merge = df_merge[df_merge['mid_td'].dt.days <= 7]

In [9]:
# # 저장
# df_merge.loc[df_merge['이동방법']== '집단']['중간집결지 주소'].value_counts().to_csv('중간집결지개수및인원.csv', encoding = 'cp949')

In [10]:
# pattern = r'^(.*?(?:도|특별시|광역시|특별자치도)\s*[^\s]+?(?:시|군|구))'
pattern = r'^(세종특별자치시|.*?(?:도|특별시|광역시|특별자치도)\s*[^\s]+?(?:시|군|구))'

def extract_province_city(addr):
    m = re.match(pattern, addr)
    return m.group(1) if m else '도시 패턴없음'

df_merge["출발지_도시단위"] = df_merge["출발지 주소"].apply(extract_province_city)
df_merge["중간집결지_도시단위"] = df_merge["중간집결지 주소"].apply(extract_province_city)
df_merge["최종도착지_도시단위"] = df_merge["최종 도착지 주소"].apply(extract_province_city)

print(df_merge)

            순번                   출발지 주소                   중간집결지 주소  \
0            1  경기도 고양시 덕양구 중앙로633번길 12        경기도 고양시 덕양구 혜음로 142   
2            3   서울특별시 양천구 목동중앙본로11길 19       서울특별시 서초구 사평대로45길 26   
3            4        서울특별시 강남구 학동로 432            경기도 과천시 중앙로 294   
4            5         대구광역시 남구 명덕로 236           대구광역시 수성구 무열로 56   
6            7   경상남도 창원시 성산구 외동반림로 219  경상남도 창원시 성산구 중앙대로250번길 13   
...        ...                      ...                        ...   
874978  880063    강원특별자치도 원주시 단관공원길 111        강원특별자치도 원주시 상지대길 83   
874979  880064    강원특별자치도 원주시 단관공원길 111        강원특별자치도 원주시 상지대길 83   
874980  880065    강원특별자치도 원주시 단관공원길 111        강원특별자치도 원주시 상지대길 83   
874981  880066    강원특별자치도 원주시 단관공원길 111        강원특별자치도 원주시 상지대길 83   
874982  880067    강원특별자치도 원주시 단관공원길 111        강원특별자치도 원주시 상지대길 83   

                        최종 도착지 주소 이동방법  start_lat   start_lng    mid_lat  \
0             경기도 고양시 덕양구 혜음로 142   개별  37.626949  126.827581  37.713020   
2      

In [11]:
print(df_merge.loc[df_merge["출발지_도시단위"] == '도시 패턴없음'])
print(df_merge.loc[df_merge["중간집결지_도시단위"] == '도시 패턴없음'])
print(df_merge.loc[df_merge["최종도착지_도시단위"] == '도시 패턴없음'])

Empty DataFrame
Columns: [순번, 출발지 주소, 중간집결지 주소, 최종 도착지 주소, 이동방법, start_lat, start_lng, mid_lat, mid_lng, end_lat, end_lng, mid_td, end_td, 출발지_도시단위, 중간집결지_도시단위, 최종도착지_도시단위]
Index: []
Empty DataFrame
Columns: [순번, 출발지 주소, 중간집결지 주소, 최종 도착지 주소, 이동방법, start_lat, start_lng, mid_lat, mid_lng, end_lat, end_lng, mid_td, end_td, 출발지_도시단위, 중간집결지_도시단위, 최종도착지_도시단위]
Index: []
Empty DataFrame
Columns: [순번, 출발지 주소, 중간집결지 주소, 최종 도착지 주소, 이동방법, start_lat, start_lng, mid_lat, mid_lng, end_lat, end_lng, mid_td, end_td, 출발지_도시단위, 중간집결지_도시단위, 최종도착지_도시단위]
Index: []


In [12]:
# df_solo = df_merge[df_merge['이동방법'] == '개별']
# df_grp = df_merge[df_merge['이동방법'] == '집단']

In [13]:
# solo_start_cnt = pd.DataFrame(df_solo['출발지_도시단위'].value_counts()).reset_index()
# solo_end_cnt = pd.DataFrame(df_solo['최종도착지_도시단위'].value_counts()).reset_index()

# solo_start_cnt = solo_start_cnt.rename(columns = {'출발지_도시단위': '도시', 'count':'출발지_cnt'})
# solo_end_cnt = solo_end_cnt.rename(columns = {'최종도착지_도시단위': '도시', 'count':'도착지_cnt'})

# solo_merge = pd.merge(solo_start_cnt, solo_end_cnt, on = '도시', how = 'outer').fillna(0).reset_index(drop=True)
# solo_merge

In [14]:
# solo_merge.style.background_gradient(cmap='pink_r', axis = None)

In [15]:
# from geopy.geocoders import Nominatim

# geolocator = Nominatim(user_agent="geo_korea")

# lat_solo = []
# lng_solo = []

# city_solo = list(solo_merge['도시'].unique())

# for idx in city_solo:
#     location = geolocator.geocode(idx)
#     lat_solo.append(location.latitude)
#     lng_solo.append(location.longitude)

#     print((location.latitude, location.longitude))

In [16]:
# solo_merge_latlng = pd.concat([solo_merge,pd.Series(lat_solo), pd.Series(lng_solo)], axis = 1)
# solo_merge_latlng = solo_merge_latlng.rename(columns = {0:'위도', 1: '경도'})
# solo_merge_latlng

In [17]:
# group_start_cnt = pd.DataFrame(df_grp['출발지_도시단위'].value_counts()).reset_index()
# group_mid_cnt = pd.DataFrame(df_grp['중간집결지_도시단위'].value_counts()).reset_index()
# group_end_cnt = pd.DataFrame(df_grp['최종도착지_도시단위'].value_counts()).reset_index()

# group_start_cnt = group_start_cnt.rename(columns = {'출발지_도시단위': '도시', 'count':'출발지_cnt'})
# group_mid_cnt = group_mid_cnt.rename(columns = {'중간집결지_도시단위': '도시', 'count':'중간지_cnt'})
# group_end_cnt = group_end_cnt.rename(columns = {'최종도착지_도시단위': '도시', 'count':'도착지_cnt'})

# group_merge = pd.merge(group_start_cnt, group_mid_cnt, on = '도시', how = 'outer').fillna(0).reset_index(drop=True)
# group_merge = pd.merge(group_merge, group_end_cnt, on = '도시', how = 'outer').fillna(0).reset_index(drop=True)
# group_merge

# group_merge.style.background_gradient(cmap='pink_r', axis = None)

# from geopy.geocoders import Nominatim

# geolocator = Nominatim(user_agent="geo_korea")

# lat_group = []
# lng_group = []

# city_group = list(group_merge['도시'].unique())

# for idx in city_group:
#     location = geolocator.geocode(idx)
#     lat_group.append(location.latitude)
#     lng_group.append(location.longitude)

#     print((location.latitude, location.longitude))

# group_merge_latlng = pd.concat([group_merge,pd.Series(lat_group), pd.Series(lng_group)], axis = 1)
# group_merge_latlng = group_merge_latlng.rename(columns = {0:'위도', 1: '경도'})
# group_merge_latlng

In [18]:
# # 개인 - 출발
# m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')
# m.add_child(plugins.HeatMap(zip(solo_merge_latlng['위도'], solo_merge_latlng['경도'], solo_merge_latlng['출발지_cnt'])))
# m.save("solo_merge_start.html")

# # 개인 - 도착
# m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')
# m.add_child(plugins.HeatMap(zip(solo_merge_latlng['위도'], solo_merge_latlng['경도'], solo_merge_latlng['도착지_cnt'])))
# m.save("solo_merge_end.html")

In [19]:
# # 그룹 - 출발
# m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')
# m.add_child(plugins.HeatMap(zip(group_merge_latlng['위도'], group_merge_latlng['경도'], group_merge_latlng['출발지_cnt'])))
# m.save("group_merge_start.html")

# # 그룹 - 중간,
# m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')
# m.add_child(plugins.HeatMap(zip(group_merge_latlng['위도'], group_merge_latlng['경도'], group_merge_latlng['중간지_cnt'])))
# m.save("group_merge_mid.html")

# # 그룹 - 도착
# m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')
# m.add_child(plugins.HeatMap(zip(group_merge_latlng['위도'], group_merge_latlng['경도'], group_merge_latlng['도착지_cnt'])))
# m.save("group_merge_end.html")

In [20]:
start_cnt = pd.DataFrame(df_merge['출발지_도시단위'].value_counts()).reset_index()
mid_cnt = pd.DataFrame(df_merge['중간집결지_도시단위'].value_counts()).reset_index()
end_cnt = pd.DataFrame(df_merge['최종도착지_도시단위'].value_counts()).reset_index()

start_cnt = start_cnt.rename(columns = {'출발지_도시단위': '도시', 'count':'출발지_cnt'})
mid_cnt = mid_cnt.rename(columns = {'중간집결지_도시단위': '도시', 'count':'중간지_cnt'})
end_cnt = end_cnt.rename(columns = {'최종도착지_도시단위': '도시', 'count':'도착지_cnt'})

merge = pd.merge(start_cnt, mid_cnt, on = '도시', how = 'outer').fillna(0).reset_index(drop=True)
merge = pd.merge(merge, end_cnt, on = '도시', how = 'outer').fillna(0).reset_index(drop=True)
merge

merge.style.background_gradient(cmap='pink_r', axis = None)

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geo_korea")

lat = []
lng = []

city = list(merge['도시'].unique())

for idx in city:
    location = geolocator.geocode(idx)
    lat.append(location.latitude)
    lng.append(location.longitude)

    print((location.latitude, location.longitude))

merge_latlng = pd.concat([merge,pd.Series(lat), pd.Series(lng)], axis = 1)
merge_latlng = merge_latlng.rename(columns = {0:'위도', 1: '경도'})
merge_latlng

(38.6478743, 128.2575748)
(38.2069574, 128.5913424)
(37.3420996, 127.9197603)
(38.1468023, 127.3133)
(37.8810569, 127.7297641)
(37.1637943, 128.9857176)
(38.1063, 127.708)
(37.7525313, 128.8759523)
(38.3808, 128.4678)
(37.5245142, 129.1146302)
(37.4498745, 129.1652969)
(38.2069574, 128.5913424)
(38.1055376, 127.9910471)
(38.0751, 128.6191)
(37.1834983, 128.4619814)
(37.3420996, 127.9197603)
(38.0694, 128.1706)
(37.380456, 128.6609485)
(38.1468023, 127.3133)
(37.8810569, 127.7297641)
(37.1637943, 128.9857176)
(37.3705, 128.3903)
(37.6967, 127.8888)
(38.1063, 127.708)
(37.4916, 127.985)
(37.8314, 127.5098)
(37.6172511, 126.8363154)
(37.4287219, 127.0021206)
(37.4785475, 126.8643937)
(37.424105, 127.376995)
(37.5888053, 127.1609126)
(37.3615229, 126.9349025)
(37.6155683, 126.7157798)
(37.6359398, 127.2165051)
(37.8930425, 127.0578117)
(37.5288774, 126.7937471)
(37.3866554, 127.1201242)
(37.2633325, 127.0287472)
(37.3395928, 126.7570649)
(37.3217152, 126.8308604)
(37.013574, 127.329939)
(3

,도시,출발지_cnt,중간지_cnt,도착지_cnt,위도,경도
0,강원도 고성군,1,0.0,0.0,38.647874,128.257575
1,강원도 속초시,1,0.0,0.0,38.206957,128.591342
2,강원도 원주시,5,0.0,0.0,37.342100,127.919760
3,강원도 철원군,1,0.0,0.0,38.146802,127.313300
4,강원도 춘천시,4,0.0,0.0,37.881057,127.729764
...,...,...,...,...,...,...
231,충청북도 제천시,1579,1403.0,786.0,37.132604,128.191351
232,충청북도 증평군,345,6745.0,7486.0,36.785213,127.581816
233,충청북도 진천군,871,1086.0,1086.0,36.855461,127.435393
234,충청북도 청주시,11942,7275.0,4945.0,36.642117,127.489174


In [21]:
merge_latlng.loc[merge_latlng['도시']=='경기도 남양주시']

,도시,출발지_cnt,중간지_cnt,도착지_cnt,위도,경도
33,경기도 남양주시,22832,30586.0,41323.0,37.63594,127.216505


In [22]:
# 전체 - 출발
m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')
m.add_child(plugins.HeatMap(zip(merge_latlng['위도'], merge_latlng['경도'], merge_latlng['출발지_cnt'])))
m.save("merge_start1.html")

# 전체 - 중간
m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')
m.add_child(plugins.HeatMap(zip(merge_latlng['위도'], merge_latlng['경도'], merge_latlng['중간지_cnt'])))
m.save("merge_mid1.html")

# 전체 - 도착
m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')
m.add_child(plugins.HeatMap(zip(merge_latlng['위도'], merge_latlng['경도'], merge_latlng['도착지_cnt'])))
m.save("merge_end1.html")